In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

from ionpy.analysis import ResultsLoader
from ese.experiment.experiment import CalibrationExperiment
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

In [ ]:
# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path = f"{root}/WMH_aug_runs"

dfc = rs.load_configs(
    path,
    properties=False,
)

df = rs.load_metrics(dfc)

In [ ]:
best_exp = rs.get_experiment(
    df=df,
    exp_class=CalibrationExperiment,
    metric="val-dice_score",
    checkpoint="max-val-dice_score",
    device="cuda"
)

In [ ]:
%%yaml dataset_cfg 

dataset:
    _class: ese.experiment.datasets.WMH
    annotator: observer_o12
    axis: 0
    split: '?'
    slicing: full 
    task: '?'
    version: 0.2

In [ ]:
dataset_options = {
    'dataset.split': ['train', 'val', 'cal'],
    'dataset.task': ['Singapore', 'Amsterdam']
}

In [ ]:
from ionpy.util import dict_product, Config

base_cfg = Config(dataset_cfg)

cfgs = []
for cfg_update in dict_product(dataset_options):
    cfg = base_cfg.update(cfg_update)
    cfgs.append(cfg)

In [ ]:
len(cfgs)

In [ ]:
cfgs[0]

In [ ]:
from ese.experiment.analysis.inference import get_dice_breakdown

get_dice_breakdown(best_exp, cfgs)